A demonstration of various VSA structures converted to C programming from its original MATLAB source code. Implementation serves strictly as a demonstration but will not run in the current environment and has not been optimized for testing.

**seq_slam function:**

seqSLAMConv function is an integral part of the SEQUENCE-SLAM algorithm, which is used for identifying similar sequences in a database of images or patterns.

It takes a difference matrix—where each element represents the dissimilarity between two images—and a sequence length as inputs. The function then applies a series of convolutions across varying velocity scales to this difference matrix.

These convolutions are designed to enhance the similarity of sequential image matches over isolated matches. The velocity scale adjusts the convolution filter to account for varying motion speeds in the sequences.

After applying the convolutions, the function normalizes the results and combines them to produce a new difference matrix that emphasizes the strongest sequence matches.

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// Function prototypes
int isSame(double **X, double **Y, int rows, int cols);
double **getFilter(int n, double v);
double **imfilter(double **DD, double **H, int rows, int cols, double supVal);
double **ones(int rows, int cols);
void printMatrix(double **matrix, int rows, int cols);
void freeMatrix(double **matrix, int rows);

// Approximated SeqSLAM using convolutions
double **seqSLAMConv(double **DD, int n) {
    // setup possible velocities
    double *v;
    int numV;
    if (n == 1) {
        v = (double *)malloc(sizeof(double));
        v[0] = 1;
        numV = 1;
    } else {
        v = (double *)malloc((n - 1) * sizeof(double));
        double start = 0.8;
        double end = 1.2;
        double step = (end - start) / (n - 1);
        for (int i = 0; i < n - 1; i++) {
            v[i] = start + i * step;
        }
        numV = n - 1;
    }

    // compute and apply filter masks
    int rows = sizeof(DD) / sizeof(DD[0]);
    int cols = sizeof(DD[0]) / sizeof(DD[0][0]);
    double ***T = (double ***)malloc(rows * sizeof(double **));
    double supVal = 0; // abs(max(DD(:))) * 1.1;
    double **prevFilter = NULL;
    int idx = 0;
    for (int i = 0; i < numV; i++) {
        double **H = getFilter(n, v[i]);
        if (!isSame(H, prevFilter, n, n)) {
            T[idx] = imfilter(DD, H, rows, cols, supVal);
            // weight to account for boundaries
            double **W = imfilter(ones(rows, cols), H, rows, cols, 0);
            for (int j = 0; j < rows; j++) {
                for (int k = 0; k < cols; k++) {
                    T[idx][j][k] /= W[j][k];
                }
            }
            idx++;
            prevFilter = H;
        }
    }
    int numT = idx;
    double **DDD = (double **)malloc(rows * sizeof(double *));
    for (int i = 0; i < rows; i++) {
        DDD[i] = (double *)malloc(cols * sizeof(double));
        for (int j = 0; j < cols; j++) {
            double maxVal = T[0][i][j];
            for (int k = 1; k < numT; k++) {
                if (T[k][i][j] > maxVal) {
                    maxVal = T[k][i][j];
                }
            }
            DDD[i][j] = maxVal;
        }
    }

    // Free memory
    free(v);
    for (int i = 0; i < numT; i++) {
        freeMatrix(T[i], rows);
    }
    free(T);
    freeMatrix(prevFilter, n);
    freeMatrix(W, rows);

    return DDD;
}

// Generate a [~v*n, n] mask H with a line of 1 with slope v. Dimensions of
// H are odd, the line goes through the center element.
// n ... number of ones
// v ... slope
double **getFilter(int n, double v) {
    assert(v > 0);
    assert(v >= 0.5);
    int nh = floor(n / 2);
    // get bottom right part
    double *x = (double *)malloc(nh * sizeof(double));
    double *y = (double *)malloc(nh * sizeof(double));
    for (int i = 0; i < nh; i++) {
        x[i] = i + 1;
        y[i] = round(x[i] * v);
    }
    int hh = y[nh - 1];
    int wh = x[nh - 1];
    double **Hh = (double **)malloc(hh * sizeof(double *));
    for (int i = 0; i < hh; i++) {
        Hh[i] = (double *)malloc(wh * sizeof(double));
        for (int j = 0; j < wh; j++) {
            Hh[i][j] = 0;
        }
    }
    for (int i = 0; i < nh; i++) {
        Hh[y[i] - 1][x[i] - 1] = 1;
    }
    // combine: top-left is Hh, centre is 1, bottom-right is Hh
    double **H = (double **)malloc((2 * hh + 1) * sizeof(double *));
    for (int i = 0; i < 2 * hh + 1; i++) {
        H[i] = (double *)malloc((wh + 1) * sizeof(double));
        for (int j = 0; j < wh + 1; j++) {
            H[i][j] = 0;
        }
    }
    for (int i = 0; i < hh; i++) {
        for (int j = 0; j < wh; j++) {
            H[i][j] = Hh[i][j];
        }
    }
    H[hh][wh / 2] = 1;
    for (int i = 0; i < hh; i++) {
        for (int j = 0; j < wh; j++) {
            H[i + hh + 1][j] = Hh[i][j];
        }
    }

    // Free memory
    free(x);
    free(y);
    for (int i = 0; i < hh; i++) {
        free(Hh[i]);
    }
    free(Hh);

    return H;
}

// Are matrices X and Y identical?
int isSame(double **X, double **Y, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            if (X[i][j] != Y[i][j]) {
                return 0;
            }
        }
    }
    return 1;
}

// Apply filter H to matrix DD using convolution
double **imfilter(double **DD, double **H, int rows, int cols, double supVal) {
    int nh = sizeof(H) / sizeof(H[0]);
    int wh = sizeof(H[0]) / sizeof(H[0][0]);
    int h = floor(nh / 2);
    int w = floor(wh / 2);
    double **result = (double **)malloc(rows * sizeof(double *));
    for (int i = 0; i < rows; i++) {
        result[i] = (double *)malloc(cols * sizeof(double));
        for (int j = 0; j < cols; j++) {
            result[i][j] = 0;
        }
    }
    for (int i = h; i < rows - h; i++) {
        for (int j = w; j < cols - w; j++) {
            for (int k = -h; k <= h; k++) {
                for (int l = -w; l <= w; l++) {
                    result[i][j] += DD[i + k][j + l] * H[k + h][l + w];
                }
            }
            if (result[i][j] > supVal) {
                result[i][j] = supVal;
            }
        }
    }
    return result;
}

// Create a matrix of ones
double **ones(int rows, int cols) {
    double **matrix = (double **)malloc(rows * sizeof(double *));
    for (int i = 0; i < rows; i++) {
        matrix[i] = (double *)malloc(cols * sizeof(double));
        for (int j = 0; j < cols; j++) {
            matrix[i][j] = 1;
        }
    }
    return matrix;
}

// Print matrix
void printMatrix(double **matrix, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%f ", matrix[i][j]);
        }
        printf("\n");
    }
}

// Free matrix memory
void freeMatrix(double **matrix, int rows) {
    for (int i = 0; i < rows; i++) {
        free(matrix[i]);
    }
    free(matrix);
}

int main() {
    // Test the seqSLAMConv function
    int rows = 5;
    int cols = 5;
    double **DD = (double **)malloc(rows * sizeof(double *));
    for (int i = 0; i < rows; i++) {
        DD[i] = (double *)malloc(cols * sizeof(double));
        for (int j = 0; j < cols; j++) {
            DD[i][j] = i + j;
        }
    }
    double **DDD = seqSLAMConv(DD, 3);
    printMatrix(DDD, rows, cols);

    // Free memory
    freeMatrix(DD, rows);
    freeMatrix(DDD, rows);

    return 0;
}

**get_slsbh function:**


The get_sLSBH function is designed to generate Sparse Localized Spectral Binary Hashes (sLSBH) for a given dataset Y.

The process begins by determining a threshold n based on a sparsity parameter s, which dictates the number of significant components to retain. The function first performs a random projection of the data and then sorts it to identify the top n components, thereby sparsifying the data.

This operation is performed twice—once in descending order to create a hash L1 and once in ascending order to create L2. These two hashes are then concatenated to form a more robust binary hash L, which captures the essential features of the dataset while reducing its dimensionality.

This binary hash can be used in various tasks such as indexing, retrieval, and classification, providing a compressed yet informative representation of the data.

In [ ]:
#include <stdio.h>
#include <stdlib.h>

float** get_sLSBH(float** Y, float s, int m, int n) {
  int i, j, k;
  int num_features = n;
  int num_descriptors = m;
  int num_selected_features = num_features * s;

  // random projection
  float** Y2 = (float**)malloc(num_descriptors * sizeof(float*));
  for (i = 0; i < num_descriptors; i++) {
    Y2[i] = (float*)malloc(num_features * sizeof(float));
    for (j = 0; j < num_features; j++) {
      Y2[i][j] = Y[i][j];
    }
  }

  // sort
  int** IDX = (int**)malloc(num_descriptors * sizeof(int*));
  for (i = 0; i < num_descriptors; i++) {
    IDX[i] = (int*)malloc(num_features * sizeof(int));
    for (j = 0; j < num_features; j++) {
      IDX[i][j] = j;
    }
    for (j = 0; j < num_features - 1; j++) {
      for (k = 0; k < num_features - j - 1; k++) {
        if (Y2[i][k] < Y2[i][k + 1]) {
          float temp = Y2[i][k];
          Y2[i][k] = Y2[i][k + 1];
          Y2[i][k + 1] = temp;

          int temp_idx = IDX[i][k];
          IDX[i][k] = IDX[i][k + 1];
          IDX[i][k + 1] = temp_idx;
        }
      }
    }
  }

  // sparsification
  float** L1 = (float**)malloc(num_descriptors * sizeof(float*));
  for (i = 0; i < num_descriptors; i++) {
    L1[i] = (float*)malloc(num_features * sizeof(float));
    for (j = 0; j < num_features; j++) {
      L1[i][j] = 0;
    }
    for (j = 0; j < num_selected_features; j++) {
      L1[i][IDX[i][j]] = 1;
    }
  }

  // sort
  for (i = 0; i < num_descriptors; i++) {
    for (j = 0; j < num_features - 1; j++) {
      for (k = 0; k < num_features - j - 1; k++) {
        if (Y2[i][k] > Y2[i][k + 1]) {
          float temp = Y2[i][k];
          Y2[i][k] = Y2[i][k + 1];
          Y2[i][k + 1] = temp;

          int temp_idx = IDX[i][k];
          IDX[i][k] = IDX[i][k + 1];
          IDX[i][k + 1] = temp_idx;
        }
      }
    }
  }

  // sparsification
  float** L2 = (float**)malloc(num_descriptors * sizeof(float*));
  for (i = 0; i < num_descriptors; i++) {
    L2[i] = (float*)malloc(num_features * sizeof(float));
    for (j = 0; j < num_features; j++) {
      L2[i][j] = 0;
    }
    for (j = 0; j < num_selected_features; j++) {
      L2[i][IDX[i][j]] = 1;
    }
  }

  // concat
  float** L = (float**)malloc(num_descriptors * sizeof(float*));
  for (i = 0; i < num_descriptors; i++) {
    L[i] = (float*)malloc((2 * num_features) * sizeof(float));
    for (j = 0; j < num_features; j++) {
      L[i][j] = L1[i][j];
    }
    for (j = 0; j < num_features; j++) {
      L[i][j + num_features] = L2[i][j];
    }
  }

  return L;
}

**creatPR function:**


The createPR function generates precision-recall (PR) curves for evaluating sequence matching algorithms. Given a similarity matrix S, hard ground truth GThard, and optionally a soft ground truth GTsoft, it calculates the precision and recall for different threshold values.

If the removeDiagFlag is set, diagonal elements representing self-matches are removed from consideration. The singleMatchFlag ensures only the highest similarity scores are considered for each sequence, effectively enforcing a one-to-one match.

The function iteratively calculates precision and recall across the range of scores in S, identifying the threshold that yields the best F1 score. Additionally, the visualizePRAtThresh helper function creates a visual representation of true positives, false negatives, and false positives at the optimal threshold, aiding in the interpretability of the results.

In [ ]:
#include <stdio.h>
#include <stdlib.h>

typedef struct {
    int rows;
    int cols;
    double *data;
} Matrix;

Matrix createMatrix(int rows, int cols) {
    Matrix mat;
    mat.rows = rows;
    mat.cols = cols;
    mat.data = (double *)malloc(rows * cols * sizeof(double));
    return mat;
}

void freeMatrix(Matrix mat) {
    free(mat.data);
}

Matrix zeros(int rows, int cols) {
    Matrix mat = createMatrix(rows, cols);
    for (int i = 0; i < rows * cols; i++) {
        mat.data[i] = 0.0;
    }
    return mat;
}

Matrix eye(int n) {
    Matrix mat = zeros(n, n);
    for (int i = 0; i < n; i++) {
        mat.data[i * n + i] = 1.0;
    }
    return mat;
}

Matrix copyMatrix(Matrix mat) {
    Matrix copy = createMatrix(mat.rows, mat.cols);
    for (int i = 0; i < mat.rows * mat.cols; i++) {
        copy.data[i] = mat.data[i];
    }
    return copy;
}

Matrix max(Matrix mat1, Matrix mat2) {
    Matrix result = createMatrix(mat1.rows, mat1.cols);
    for (int i = 0; i < mat1.rows * mat1.cols; i++) {
        result.data[i] = (mat1.data[i] > mat2.data[i]) ? mat1.data[i] : mat2.data[i];
    }
    return result;
}

Matrix min(Matrix mat1, Matrix mat2) {
    Matrix result = createMatrix(mat1.rows, mat1.cols);
    for (int i = 0; i < mat1.rows * mat1.cols; i++) {
        result.data[i] = (mat1.data[i] < mat2.data[i]) ? mat1.data[i] : mat2.data[i];
    }
    return result;
}

Matrix logical(Matrix mat) {
    Matrix result = createMatrix(mat.rows, mat.cols);
    for (int i = 0; i < mat.rows * mat.cols; i++) {
        result.data[i] = (mat.data[i] != 0.0) ? 1.0 : 0.0;
    }
    return result;
}

Matrix linspace(double start, double end, int num) {
    Matrix mat = createMatrix(1, num);
    double step = (end - start) / (num - 1);
    double value = start;
    for (int i = 0; i < num; i++) {
        mat.data[i] = value;
        value += step;
    }
    return mat;
}

Matrix imreconstruct(Matrix seedIm, Matrix mask) {
    Matrix result = copyMatrix(mask);
    int changed = 1;
    while (changed) {
        changed = 0;
        for (int i = 0; i < result.rows * result.cols; i++) {
            if (seedIm.data[i] > result.data[i]) {
                result.data[i] = seedIm.data[i];
                changed = 1;
            }
        }
    }
    return result;
}

int nnz(Matrix mat) {
    int count = 0;
    for (int i = 0; i < mat.rows * mat.cols; i++) {
        if (mat.data[i] != 0.0) {
            count++;
        }
    }
    return count;
}

Matrix visualizePRAtThresh(Matrix S, Matrix GT, double t) {
    Matrix B = createMatrix(S.rows, S.cols);
    for (int i = 0; i < S.rows * S.cols; i++) {
        B.data[i] = (S.data[i] >= t) ? 1.0 : 0.0;
    }
    Matrix TP = createMatrix(GT.rows, GT.cols);
    Matrix FN = createMatrix(GT.rows, GT.cols);
    Matrix FP = createMatrix(GT.rows, GT.cols);
    for (int i = 0; i < GT.rows * GT.cols; i++) {
        TP.data[i] = (GT.data[i] != 0.0 && B.data[i] != 0.0) ? 1.0 : 0.0;
        FN.data[i] = (GT.data[i] != 0.0 && B.data[i] == 0.0) ? 1.0 : 0.0;
        FP.data[i] = (GT.data[i] == 0.0 && B.data[i] != 0.0) ? 1.0 : 0.0;
    }
    Matrix V = createMatrix(GT.rows, GT.cols * 3);
    for (int i = 0; i < GT.rows * GT.cols; i++) {
        V.data[i] = FP.data[i];
        V.data[GT.rows * GT.cols + i] = TP.data[i];
        V.data[2 * GT.rows * GT.cols + i] = FN.data[i];
    }
    freeMatrix(B);
    freeMatrix(TP);
    freeMatrix(FN);
    freeMatrix(FP);
    return V;
}

void createPR(Matrix S, Matrix GThard, Matrix GTsoft, int removeDiagFlag, int singleMatchFlag, int evalAllSValuesFlag, Matrix *P, Matrix *R, Matrix *V, double *bestP, double *bestR, double *bestF) {
    if (!evalAllSValuesFlag) {
        evalAllSValuesFlag = 0;
    }
    Matrix GT = logical(GThard);
    if (removeDiagFlag) {
        Matrix maindiaMask;
        if (GTsoft.rows > 0 && GTsoft.cols > 0) {
            Matrix seedIm = zeros(GT.rows, GT.cols);
            seedIm.data[0] = 1.0;
            maindiaMask = logical(imreconstruct(seedIm, GTsoft));
            Matrix transposeMaindiaMask = createMatrix(maindiaMask.cols, maindiaMask.rows);
            for (int i = 0; i < maindiaMask.rows; i++) {
                for (int j = 0; j < maindiaMask.cols; j++) {
                    transposeMaindiaMask.data[j * maindiaMask.rows + i] = maindiaMask.data[i * maindiaMask.cols + j];
                }
            }
            maindiaMask = max(maindiaMask, transposeMaindiaMask);
            freeMatrix(transposeMaindiaMask);
        } else {
            maindiaMask = eye(GT.rows);
        }
        Matrix lowerTriangle = createMatrix(S.rows, S.cols);
        for (int i = 0; i < S.rows; i++) {
            for (int j = 0; j < S.cols; j++) {
                lowerTriangle.data[i * S.cols + j] = (i >= j) ? 1.0 : 0.0;
            }
        }
        maindiaMask = max(maindiaMask, lowerTriangle);
        Matrix S_copy = copyMatrix(S);
        for (int i = 0; i < S.rows * S.cols; i++) {
            if (maindiaMask.data[i] != 0.0) {
                S_copy.data[i] = S_copy.data[0];
            }
        }
        freeMatrix(maindiaMask);
        freeMatrix(lowerTriangle);
        freeMatrix(S);
        S = S_copy;
        freeMatrix(S_copy);
        for (int i = 0; i < GT.rows * GT.cols; i++) {
            if (GTsoft.data[i] != 0.0 && GThard.data[i] == 0.0) {
                S.data[i] = S.data[0];
            }
        }
    }
    if (singleMatchFlag) {
        Matrix hIdx = createMatrix(1, S.cols);
        for (int j = 0; j < S.cols; j++) {
            double maxVal = S.data[j];
            int maxIdx = 0;
            for (int i = 1; i < S.rows; i++) {
                if (S.data[i * S.cols + j] > maxVal) {
                    maxVal = S.data[i * S.cols + j];
                    maxIdx = i;
                }
            }
            hIdx.data[j] = maxIdx;
        }
        Matrix T = createMatrix(S.rows, S.cols);
        for (int i = 0; i < S.rows * S.cols; i++) {
            T.data[i] = S.data[0];
        }
        for (int j = 0; j < S.cols; j++) {
            int hIdxInd = hIdx.data[j] * S.cols + j;
            T.data[hIdxInd] = S.data[hIdxInd];
        }
        freeMatrix(S);
        S = T;
        freeMatrix(T);
        freeMatrix(hIdx);
    }
    Matrix R_temp = createMatrix(1, 1);
    Matrix P_temp = createMatrix(1, 1);
    double bestF_temp = 0.0;
    double bestT = S.data[0];
    double bestP_temp = 0.0;
    double bestR_temp = 0.0;
    Matrix s_vals;
    if (evalAllSValuesFlag) {
        s_vals = linspace(S.data[S.rows * S.cols - 1], S.data[0], S.rows * S.cols);
    } else {
        s_vals = linspace(S.data[0], S.data[S.rows * S.cols - 1], 100);
    }
    Matrix GT_sparse = logical(GT);
    for (int i = 0; i < s_vals.cols; i++) {
        double threshold = s_vals.data[i];
        Matrix B = createMatrix(S.rows, S.cols);
        for (int j = 0; j < S.rows * S.cols; j++) {
            B.data[j] = (S.data[j] >= threshold) ? 1.0 : 0.0;
        }
        int TP = nnz(max(GT_sparse, B));
        int FN = nnz(max(GT_sparse, min(1.0, -B)));
        int FP = nnz(max(min(1.0, -GT_sparse), B));
        P_temp.data[0] = (TP + FP > 0) ? (double)TP / (TP + FP) : 0.0;
        R_temp.data[0] = (TP + FN > 0) ? (double)TP / (TP + FN) : 0.0;
        double F = (P_temp.data[0] + R_temp.data[0] > 0) ? 2.0 * P_temp.data[0] * R_temp.data[0] / (P_temp.data[0] + R_temp.data[0]) : 0.0;
        if (F > bestF_temp) {
            bestF_temp = F;
            bestT = threshold;
            bestP_temp = P_temp.data[0];
            bestR_temp = R_temp.data[0];
        }
        freeMatrix(B);
    }
    freeMatrix(R_temp);
    freeMatrix(P_temp);
    R_temp = createMatrix(1, R_temp.cols + 1);
    P_temp = createMatrix(1, P_temp.cols + 1);
    for (int i = 0; i < R_temp.cols - 1; i++) {
        R_temp.data[i] = R_temp.data[i + 1];
    }
    for (int i = 0; i < P_temp.cols - 1; i++) {
        P_temp.data[i] = P_temp.data[i + 1];
    }
    R_temp.data[R_temp.cols - 1] = 1.0;
    P_temp.data[P_temp.cols - 1] = 0.0;
    *P = copyMatrix(P_temp);
    *R = copyMatrix(R_temp);
    Matrix V_temp = visualizePRAtThresh(S, GT, bestT);
    *V = copyMatrix(V_temp);
    freeMatrix(R_temp);
    freeMatrix(P_temp);
    freeMatrix(V_temp);
    *bestP = bestP_temp;
    *bestR = bestR_temp;
    *bestF = bestF_temp;
}

int main() {
    // Example usage
    Matrix S = createMatrix(3, 3);
    S.data[0] = 0.1;
    S.data[1] = 0.2;
    S.data[2] = 0.3;
    S.data[3] = 0.4;
    S.data[4] = 0.5;
    S.data[5] = 0.6;
    S.data[6] = 0.7;
    S.data[7] = 0.8;
    S.data[8] = 0.9;
    Matrix GThard = createMatrix(3, 3);
    GThard.data[0] = 1.0;
    GThard.data[1] = 0.0;
    GThard.data[2] = 1.0;
    GThard.data[3] = 0.0;
    GThard.data[4] = 1.0;
    GThard.data[5] = 0.0;
    GThard.data[6] = 1.0;
    GThard.data[7] = 0.0;
    GThard.data[8] = 1.0;
    Matrix GTsoft = createMatrix(0, 0);
    int removeDiagFlag = 1;
    int singleMatchFlag = 1;
    int evalAllSValuesFlag = 0;
    Matrix P, R, V;
    double bestP, bestR, bestF;
    createPR(S, GThard, GTsoft, removeDiagFlag, singleMatchFlag, evalAllSValuesFlag, &P, &R, &V, &bestP, &bestR, &bestF);
    freeMatrix(S);
    freeMatrix(GThard);
    freeMatrix(GTsoft);
    freeMatrix(P
}

**unbind_vectors operation:**

The unbind_vectors function is designed to reverse the binding operation between pairs of vectors in various Vector Symbolic Architecture (VSA) models. It takes as inputs two sets of vectors and the VSA model identifier, then performs model-specific unbinding operations. For instance, in MAP models, it executes element-wise multiplication, while for BSC, it uses an efficient XOR operation. The BSDC model requires a similarity search instead of direct unbinding. For HRR and FHRR, the function applies Fourier transform-based involution and complex arithmetic, respectively. In the BSDC_SHIFT model, it involves array shifting, and for BSDC_SEG, it handles sparse segments. The function showcases the diverse nature of unbinding across different VSA models, reflecting the need for tailored approaches to effectively reverse the binding of high-dimensional vectors.

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

double** unbind_vectors(char* vsa, double** vectors_1, double** vectors_2, int num_vectors, int vector_dim, double density, double** M) {
    double** unbound_vectors = (double**)malloc(num_vectors * sizeof(double*));
    for (int i = 0; i < num_vectors; i++) {
        unbound_vectors[i] = (double*)malloc(vector_dim * sizeof(double));
    }

    if (strcmp(vsa, "MAP_B") == 0 || strcmp(vsa, "MAP_C") == 0 || strcmp(vsa, "MAP_I") == 0) {
        // elementwise multiplication
        for (int i = 0; i < num_vectors; i++) {
            for (int j = 0; j < vector_dim; j++) {
                unbound_vectors[i][j] = vectors_1[i][j] * vectors_2[i][j];
            }
        }
    }
    else if (strcmp(vsa, "BSC") == 0) {
        // efficient xor implementation for multi inputs
        for (int i = 0; i < num_vectors; i++) {
            for (int j = 0; j < vector_dim; j++) {
                unbound_vectors[i][j] = (double)((int)vectors_1[i][j] + (int)vectors_2[i][j] == 1);
            }
        }
    }
    else if (strcmp(vsa, "BSDC") == 0) {
        // find the most similar item in item_mem
        printf("There is no specific unbind operator for the selected VSA - use the finding of the most similar vectors in item memory instead!\n");
    }
    else if (strcmp(vsa, "BSDC_SHIFT") == 0) {
        // calculate the shift number (sum of all ones-index)
        double* idx = (double*)malloc(num_vectors * sizeof(double));
        for (int i = 0; i < num_vectors; i++) {
            idx[i] = 0;
            for (int j = 0; j < vector_dim; j++) {
                idx[i] += (double)(j + 1) * vectors_1[i][j];
            }
        }

        // shift each column with specific index number
        for (int i = 0; i < num_vectors; i++) {
            for (int j = 0; j < vector_dim; j++) {
                unbound_vectors[i][j] = vectors_2[i][(j - (int)idx[i] + vector_dim) % vector_dim];
            }
        }

        free(idx);
    }
    else if (strcmp(vsa, "HRR") == 0) {
        // involution as approximate inverse
        double* inverse = (double*)malloc(vector_dim * sizeof(double));
        inverse[0] = vectors_1[0][0];
        for (int i = 1; i < vector_dim; i++) {
            inverse[i] = vectors_1[i][0];
        }

        double* fft_inverse = (double*)malloc(vector_dim * sizeof(double));
        double* fft_vectors_2 = (double*)malloc(vector_dim * sizeof(double));

        for (int i = 0; i < num_vectors; i++) {
            for (int j = 0; j < vector_dim; j++) {
                fft_inverse[j] = inverse[j];
                fft_vectors_2[j] = vectors_2[i][j];
            }

            fft(fft_inverse, vector_dim);
            fft(fft_vectors_2, vector_dim);

            for (int j = 0; j < vector_dim; j++) {
                unbound_vectors[i][j] = fft_inverse[j] * fft_vectors_2[j];
            }

            ifft(unbound_vectors[i], vector_dim);
        }

        free(inverse);
        free(fft_inverse);
        free(fft_vectors_2);
    }
    else if (strcmp(vsa, "HRR_VTB") == 0) {
        double* val_x = (double*)malloc(vector_dim * sizeof(double));
        double* val_y = (double*)malloc(vector_dim * sizeof(double));
        double** V_x = (double**)malloc(vector_dim * sizeof(double*));
        for (int i = 0; i < vector_dim; i++) {
            V_x[i] = (double*)malloc(vector_dim * sizeof(double));
        }

        int sub_d = (int)sqrt(vector_dim);
        if (sub_d * sub_d != vector_dim) {
            printf("In VTB: The number of dimensions must have an even root.\n");
            return NULL;
        }

        for (int i = 0; i < num_vectors; i++) {
            for (int j = 0; j < vector_dim; j++) {
                val_x[j] = vectors_1[j][i];
                val_y[j] = vectors_2[j][i];
            }

            for (int j = 0; j < vector_dim; j++) {
                for (int k = 0; k < vector_dim; k++) {
                    if (j / sub_d == k / sub_d) {
                        V_x[j][k] = val_x[(j / sub_d) * sub_d + (k % sub_d)];
                    }
                    else {
                        V_x[j][k] = 0;
                    }
                }
            }

            for (int j = 0; j < vector_dim; j++) {
                unbound_vectors[i][j] = sqrt(sub_d) * V_x[j][j] * val_y[j];
            }
        }

        free(val_x);
        free(val_y);
        for (int i = 0; i < vector_dim; i++) {
            free(V_x[i]);
        }
        free(V_x);
    }
    else if (strcmp(vsa, "FHRR") == 0) {
        for (int i = 0; i < num_vectors; i++) {
            for (int j = 0; j < vector_dim; j++) {
                unbound_vectors[i][j] = fmod(vectors_2[i][j] - vectors_1[i][j] + M_PI, 2 * M_PI) - M_PI;
            }
        }
    }
    else if (strcmp(vsa, "BSDC_SEG") == 0) {
        int num_segments = (int)(vector_dim * density);
        int size_segments = vector_dim / num_segments;

        double** role = (double**)malloc(num_segments * size_segments * sizeof(double*));
        double** filler = (double**)malloc(num_segments * size_segments * sizeof(double*));
        for (int i = 0; i < num_segments * size_segments; i++) {
            role[i] = (double*)malloc(num_vectors * sizeof(double));
            filler[i] = (double*)malloc(num_vectors * sizeof(double));
        }

        for (int i = 0; i < num_segments * size_segments; i++) {
            for (int j = 0; j < num_vectors; j++) {
                role[i][j] = vectors_1[i][j];
                filler[i][j] = vectors_2[i][j];
            }
        }

        double* role_segments = (double*)malloc(size_segments * num_segments * num_vectors * sizeof(double));
        double* filler_segments = (double*)malloc(size_segments * num_segments * num_vectors * sizeof(double));

        for (int i = 0; i < size_segments * num_segments * num_vectors; i++) {
            role_segments[i] = role[i % (size_segments * num_segments)][i / (size_segments * num_segments)][i / (size_segments * num_segments * num_vectors)];
            filler_segments[i] = filler[i % (size_segments * num_segments)][i / (size_segments * num_segments)][i / (size_segments * num_segments * num_vectors)];
        }

        int* role_idx = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* filler_idx = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* role_rows = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* role_cols = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* role_tables = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* filler_rows = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* filler_cols = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* filler_tables = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* result_rows = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));

        for (int i = 0; i < size_segments * num_segments * num_vectors; i++) {
            role_idx[i] = (int)(i + 1);
            filler_idx[i] = (int)(i + 1);
            role_rows[i] = (int)(i % size_segments);
            role_cols[i] = (int)(i / (size_segments * num_segments));
            role_tables[i] = (int)(i / (size_segments * num_segments * num_vectors));
            filler_rows[i] = (int)(i % size_segments);
            filler_cols[i] = (int)(i / (size_segments * num_segments));
            filler_tables[i] = (int)(i / (size_segments * num_segments * num_vectors));
            result_rows[i] = (int)((filler_rows[i] - role_rows[filler_cols[i] * filler_tables[i]] - 1 + size_segments) % size_segments);
        }

        for (int i = 0; i < size_segments * num_segments * num_vectors; i++) {
            unbound_vectors[result_rows[i] + filler_cols[i] * size_segments + filler_tables[i] * size_segments * num_segments][i] = 1;
        }

        for (int i = 0; i < num_segments * size_segments; i++) {
            free(role[i]);
            free(filler[i]);
        }
        free(role);
        free(filler);
        free(role_segments);
        free(filler_segments);
        free(role_idx);
        free(filler_idx);
        free(role_rows);
        free(role_cols);
        free(role_tables);
        free(filler_rows);
        free(filler_cols);
        free(filler_tables);
        free(result_rows);
    }
    else if (strcmp(vsa, "MBAT") == 0) {
        double** M_ = (double**)malloc(vector_dim * sizeof(double*));
        for (int i = 0; i < vector_dim; i++) {
            M_[i] = (double*)malloc(vector_dim * sizeof(double));
        }

        for (int i = 0; i < num_vectors; i++) {
            for (int j = 0; j < vector_dim; j++) {
                for (int k = 0; k < vector_dim; k++) {
                    if (j == k) {
                        M_[j][k] = M[j][k];
                    }
                    else {
                        M_[j][k] = 0;
                    }
                }
            }

            for (int j = 0; j < vector_dim; j++) {
                for (int k = 0; k < vector_dim; k++) {
                    unbound_vectors[i][j] += M_[j][k] * vectors_2[i][k];
                }
            }
        }

        for (int i = 0; i < vector_dim; i++) {
            free(M_[i]);
        }
        free(M_);
    }
    else {
        printf("Representation is not defined!\n");
    }

    return unbound_vectors;
}

**get_ngram operation:**


The getNgram function constructs n-grams using a specified Vector Symbolic Architecture (VSA) model. An n-gram is a contiguous sequence of n items from a given sequence of text or speech. This function translates a sequence of keys into their corresponding character codes, then iteratively binds and permutes character vectors from an item memory to form the n-gram representation. It demonstrates how sequences, such as words or phrases, can be encoded into the high-dimensional vector space characteristic of VSAs, preserving the sequential nature of the input through the VSA's binding and permutation operations.

In [ ]:
#include <stdio.h>
#include <stdlib.h>

int* getNgram(int* VSA, char* keys, int** char_item_memory, int** seq_item_memory) {
    int* key_codes = (int*)malloc(sizeof(int) * strlen(keys));
    for (int i = 0; i < strlen(keys); i++) {
        key_codes[i] = (int)keys[i];
    }
    int num_ngrams = strlen(keys);

    // create ngrams
    int* ngram = permute(VSA, char_item_memory[0], key_codes[0]);
    for (int i = 1; i < num_ngrams; i++) {
        int* permuted = permute(VSA, char_item_memory[i], key_codes[i]);
        ngram = bind(ngram, permuted, i);
    }

    return ngram;
}

int* permute(int* VSA, int* char_item_memory, int key_code) {
    // implementation of permute function
}

int* bind(int* ngram, int* permuted, int index) {
    // implementation of bind function
}

**generate_vectors operation:**


The generate_vectors function is a versatile tool for generating high-dimensional vectors according to different Vector Symbolic Architectures (VSAs). Depending on the VSA specified, it can produce dense or sparse binary vectors, continuous uniform or normal distributions, or even complex-valued vectors.

This function allows for the customization of the vector dimensionality, density of non-zero elements in sparse representations, and the number of vectors to generate. It's particularly useful for creating the foundational vector space for various VSA operations, such as binding, bundling, and unbinding, which are essential in cognitive computing tasks. Whether for binary sparse coding, holographic reduced representations, or Fourier holographic representations, this function provides the necessary building blocks for VSA-based computations.

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

float** generate_vectors(int dim, char* vsa, int num, float density) {
    int default_dim = 10000;
    char* default_vsa = "map";
    float default_density = -1;
    int default_num = 1;

    if (dim == -1) {
        dim = default_dim;
    }
    if (vsa == NULL) {
        vsa = default_vsa;
    }
    if (num == -1) {
        num = default_num;
    }
    if (density == -1) {
        switch (vsa) {
            case "BSDC":
            case "BSDC_test":
            case "BSDC_SHIFT":
            case "BSDC_THIN":
                density = 1 / sqrt(dim);
                break;
            case "BSDC_25":
                density = 0.25;
                break;
            case "BSDC_SEG":
                density = 1 / sqrt(dim);
                break;
            default:
                density = 0.5;
                break;
        }
    }

    float** gen_vecs = (float**)malloc(dim * sizeof(float*));
    for (int i = 0; i < dim; i++) {
        gen_vecs[i] = (float*)malloc(num * sizeof(float));
    }

    if (strcmp(vsa, "MAP_B") == 0 || strcmp(vsa, "MAP_I") == 0) {
        for (int i = 0; i < dim; i++) {
            for (int j = 0; j < num; j++) {
                gen_vecs[i][j] = (float)(rand() % 2) * 2 - 1;
            }
        }
    }
    else if (strcmp(vsa, "MAP_C") == 0) {
        for (int i = 0; i < dim; i++) {
            for (int j = 0; j < num; j++) {
                gen_vecs[i][j] = ((float)rand() / RAND_MAX) * 2 - 1;
            }
        }
    }
    else if (strcmp(vsa, "BSC") == 0) {
        for (int i = 0; i < dim; i++) {
            for (int j = 0; j < num; j++) {
                gen_vecs[i][j] = (float)(rand() % 2);
            }
        }
    }
    else if (strcmp(vsa, "BSDC") == 0 || strcmp(vsa, "BSDC_SHIFT") == 0 || strcmp(vsa, "BSDC_25") == 0 || strcmp(vsa, "BSDC_THIN") == 0) {
        float** rand_values = (float**)malloc(dim * sizeof(float*));
        for (int i = 0; i < dim; i++) {
            rand_values[i] = (float*)malloc(num * sizeof(float));
        }

        for (int i = 0; i < dim; i++) {
            for (int j = 0; j < num; j++) {
                rand_values[i][j] = ((float)rand() / RAND_MAX);
            }
        }

        int k = ceil(dim * density);
        int** rows = (int**)malloc(k * sizeof(int*));
        for (int i = 0; i < k; i++) {
            rows[i] = (int*)malloc(num * sizeof(int));
        }

        for (int j = 0; j < num; j++) {
            for (int i = 0; i < k; i++) {
                int max_index = 0;
                float max_value = rand_values[0][j];
                for (int l = 1; l < dim; l++) {
                    if (rand_values[l][j] > max_value) {
                        max_index = l;
                        max_value = rand_values[l][j];
                    }
                }
                rows[i][j] = max_index;
                rand_values[max_index][j] = -1;
            }
        }

        for (int i = 0; i < dim; i++) {
            for (int j = 0; j < num; j++) {
                gen_vecs[i][j] = 0;
            }
        }

        for (int j = 0; j < num; j++) {
            for (int i = 0; i < k; i++) {
                gen_vecs[rows[i][j]][j] = 1;
            }
        }

        for (int i = 0; i < k; i++) {
            free(rows[i]);
        }
        free(rows);

        for (int i = 0; i < dim; i++) {
            free(rand_values[i]);
        }
        free(rand_values);
    }
    else if (strcmp(vsa, "BSDC_SEG") == 0) {
        int num_segments = floor(dim * density);
        int size_segments = floor(dim / num_segments);
        float*** z = (float***)malloc(size_segments * sizeof(float**));
        for (int i = 0; i < size_segments; i++) {
            z[i] = (float**)malloc(num_segments * sizeof(float*));
            for (int j = 0; j < num_segments; j++) {
                z[i][j] = (float*)malloc(num * sizeof(float));
            }
        }

        int** rand_r = (int**)malloc(num_segments * sizeof(int*));
        for (int i = 0; i < num_segments; i++) {
            rand_r[i] = (int*)malloc(num * sizeof(int));
        }

        for (int j = 0; j < num; j++) {
            for (int i = 0; i < num_segments; i++) {
                rand_r[i][j] = rand() % size_segments;
            }
        }

        int** rand_c = (int**)malloc(num_segments * sizeof(int*));
        for (int i = 0; i < num_segments; i++) {
            rand_c[i] = (int*)malloc(num * sizeof(int));
        }

        for (int j = 0; j < num; j++) {
            for (int i = 0; i < num_segments; i++) {
                rand_c[i][j] = i;
            }
        }

        int** rand_t = (int**)malloc(num_segments * sizeof(int*));
        for (int i = 0; i < num_segments; i++) {
            rand_t[i] = (int*)malloc(num * sizeof(int));
        }

        for (int j = 0; j < num; j++) {
            for (int i = 0; i < num_segments; i++) {
                rand_t[i][j] = j;
            }
        }

        for (int i = 0; i < size_segments; i++) {
            for (int j = 0; j < num_segments; j++) {
                for (int k = 0; k < num; k++) {
                    z[i][j][k] = 0;
                }
            }
        }

        for (int j = 0; j < num; j++) {
            for (int i = 0; i < num_segments; i++) {
                z[rand_r[i][j]][rand_c[i][j]][rand_t[i][j]] = 1;
            }
        }

        int d = dim - num_segments * size_segments;
        float** gen_vecs_temp = (float**)malloc((dim + d) * sizeof(float*));
        for (int i = 0; i < dim + d; i++) {
            gen_vecs_temp[i] = (float*)malloc(num * sizeof(float));
        }

        for (int i = 0; i < size_segments; i++) {
            for (int j = 0; j < num_segments; j++) {
                for (int k = 0; k < num; k++) {
                    gen_vecs_temp[i * num_segments + j][k] = z[i][j][k];
                }
            }
        }

        for (int i = size_segments * num_segments; i < dim + d; i++) {
            for (int j = 0; j < num; j++) {
                gen_vecs_temp[i][j] = 0;
            }
        }

        for (int i = 0; i < dim + d; i++) {
            for (int j = 0; j < num; j++) {
                gen_vecs[i][j] = gen_vecs_temp[i][j];
            }
        }

        for (int i = 0; i < size_segments; i++) {
            for (int j = 0; j < num_segments; j++) {
                free(z[i][j]);
            }
            free(z[i]);
        }
        free(z);

        for (int i = 0; i < num_segments; i++) {
            free(rand_r[i]);
        }
        free(rand_r);

        for (int i = 0; i < num_segments; i++) {
            free(rand_c[i]);
        }
        free(rand_c);

        for (int i = 0; i < num_segments; i++) {
            free(rand_t[i]);
        }
        free(rand_t);

        for (int i = 0; i < dim + d; i++) {
            free(gen_vecs_temp[i]);
        }
        free(gen_vecs_temp);
    }
    else if (strcmp(vsa, "HRR") == 0 || strcmp(vsa, "HRR_VTB") == 0 || strcmp(vsa, "MBAT") == 0) {
        for (int i = 0; i < dim; i++) {
            for (int j = 0; j < num; j++) {
                gen_vecs[i][j] = ((float)rand() / RAND_MAX) * sqrt(1 / dim);
            }
        }
    }
    else if (strcmp(vsa, "FHRR") == 0 || strcmp(vsa, "FHRR_fft") == 0 || strcmp(vsa, "FHRR_cos") == 0) {
        for (int i = 0; i < dim; i++) {
            for (int j = 0; j < num; j++) {
                gen_vecs[i][j] = ((float)rand() / RAND_MAX) * 2 * M_PI - M_PI;
            }
        }
    }
    else {
        printf("Representation is not defined!\n");
    }

    return gen_vecs;
}

int main() {
    int dim = -1;
    char* vsa = NULL;
    int num = -1;
    float density = -1;

    float** gen_vecs = generate_vectors(dim, vsa, num, density);

    // Print generated vectors
    for (int i = 0; i < dim; i++) {
        for (int j = 0; j < num; j++) {
            printf("%f ", gen_vecs[i][j]);
        }
        printf("\n");
    }

    // Free memory
    for (int i = 0; i < dim; i++) {
        free(gen_vecs[i]);
    }
    free(gen_vecs);

    return 0;
}

**fractional_bindings operation:**

The fractional_binding function implements fractional binding operations in various Vector Symbolic Architecture (VSA) models. Fractional binding is a technique to partially bind vectors together, allowing for more nuanced and flexible representations.

For models like FHRR (Fourier Holographic Reduced Representations), it performs modulo arithmetic on the vector elements, effectively scaling them by a factor k.

In BSDC-based models (Binary Sparse Distributed Codes) and BSC (Binary Spatter Codes), the function utilizes the Fast Fourier Transform (FFT) to raise the vectors to the power of k and then retrieves the binary representation.

This function demonstrates a key aspect of VSA - the ability to modify the traditional binding operation to capture varying degrees of association between vectors, which is vital for complex cognitive tasks.

In [ ]:
#include <stdio.h>
#include <math.h>

void fractional_binding(char* vsa, double* vector, double* k, int vector_size, int k_size, double* v) {
    int i, j;
    double values[vector_size][k_size];
    double temp_vector[vector_size];
    double temp_value;

    if (strcmp(vsa, "FHRR") == 0 || strcmp(vsa, "FHRR_fft") == 0) {
        for (i = 0; i < vector_size; i++) {
            for (j = 0; j < k_size; j++) {
                values[i][j] = vector[i] * k[j];
            }
        }
        for (i = 0; i < vector_size; i++) {
            temp_value = 0;
            for (j = 0; j < k_size; j++) {
                temp_value += values[i][j];
            }
            temp_vector[i] = fmod(temp_value, 2 * M_PI);
        }
        for (i = 0; i < vector_size; i++) {
            v[i] = temp_vector[i];
        }
    } else if (strcmp(vsa, "BSDC") == 0 || strcmp(vsa, "BSDC_SEG") == 0 || strcmp(vsa, "BSDC_SHIFT") == 0 || strcmp(vsa, "BSC") == 0 || strcmp(vsa, "BSDC_25") == 0) {
        for (i = 0; i < vector_size; i++) {
            for (j = 0; j < k_size; j++) {
                values[i][j] = pow(vector[i], k[j]);
            }
        }
        for (i = 0; i < vector_size; i++) {
            temp_value = 0;
            for (j = 0; j < k_size; j++) {
                temp_value += values[i][j];
            }
            temp_vector[i] = temp_value > 0 ? 1 : 0;
        }
        for (i = 0; i < vector_size; i++) {
            v[i] = temp_vector[i];
        }
    } else {
        for (i = 0; i < vector_size; i++) {
            for (j = 0; j < k_size; j++) {
                values[i][j] = pow(vector[i], k[j]);
            }
        }
        for (i = 0; i < vector_size; i++) {
            temp_value = 0;
            for (j = 0; j < k_size; j++) {
                temp_value += values[i][j];
            }
            temp_vector[i] = temp_value;
        }
        for (i = 0; i < vector_size; i++) {
            v[i] = temp_vector[i];
        }
    }
}

int main() {
    char vsa[10] = "FHRR";
    double vector[3] = {1.0, 2.0, 3.0};
    double k[2] = {0.5, 1.0};
    int vector_size = sizeof(vector) / sizeof(vector[0]);
    int k_size = sizeof(k) / sizeof(k[0]);
    double v[vector_size];

    fractional_binding(vsa, vector, k, vector_size, k_size, v);

    printf("v = [");
    for (int i = 0; i < vector_size; i++) {
        printf("%f ", v[i]);
    }
    printf("]\n");

    return 0;
}

**convert_vectors operation:**

The convert_vectors function is a key component in Vector Symbolic Architectures (VSAs) that adapts input vectors to suit various VSA models. This function takes an input matrix Y and a specified VSA type, then processes and converts the vectors to conform to the representation requirements of the chosen VSA.

For instance, in continuous VSAs like MAP_C, it clamps the values within a specific range, while in binary VSAs like BSC, it converts the values to binary form. The function also handles models like FHRR (Fourier Holographic Reduced Representations) by transforming vectors into their angular representation.

Additionally, for sparse binary models like BSDC, it performs sparsification based on a specified density parameter. This conversion step is crucial for ensuring that the input data is compatible with the specific operations and characteristics of the chosen VSA model.

In [ ]:
#include <stdio.h>
#include <math.h>

void convert_vectors(double *vsa, double *Y, double density, double *values, int rows, int cols) {
    int i, j;
    if (density == -1) {
        // set default density
        if (strcmp(vsa, "BSDC") == 0 || strcmp(vsa, "BSDC_test") == 0 || strcmp(vsa, "BSDC_SHIFT") == 0) {
            density = 1 / sqrt(cols);
        } else if (strcmp(vsa, "BSDC_25") == 0) {
            density = 0.25;
        } else if (strcmp(vsa, "BSDC_SEG") == 0) {
            density = 1 / sqrt(cols);
        } else {
            density = 0.5;
        }
    }
    if (strcmp(vsa, "MAP_C") == 0) {
        // convert
        for (i = 0; i < rows; i++) {
            for (j = 0; j < cols; j++) {
                if (Y[i * cols + j] > 1) {
                    values[i * cols + j] = 1;
                } else if (Y[i * cols + j] < -1) {
                    values[i * cols + j] = -1;
                } else {
                    values[i * cols + j] = Y[i * cols + j];
                }
            }
        }
    } else if (strcmp(vsa, "map_trans_uniform") == 0) {
        // convert
        for (i = 0; i < rows; i++) {
            double mean = 0;
            double var = 0;
            for (j = 0; j < cols; j++) {
                mean += Y[i * cols + j];
            }
            mean /= cols;
            for (j = 0; j < cols; j++) {
                var += pow(Y[i * cols + j] - mean, 2);
            }
            var /= cols;
            for (j = 0; j < cols; j++) {
                double cdf = 0.5 * (1 + erf((Y[i * cols + j] - mean) / (sqrt(2 * var))));
                values[i * cols + j] = 2 * cdf - 1;
            }
        }
    } else if (strcmp(vsa, "MAP_B") == 0 || strcmp(vsa, "MAP_I") == 0) {
        // convert
        for (i = 0; i < rows; i++) {
            for (j = 0; j < cols; j++) {
                values[i * cols + j] = (Y[i * cols + j] > 0) ? 1 : -1;
            }
        }
    } else if (strcmp(vsa, "BSC") == 0) {
        // convert
        for (i = 0; i < rows; i++) {
            for (j = 0; j < cols; j++) {
                values[i * cols + j] = (Y[i * cols + j] > 0) ? 1 : 0;
            }
        }
    } else if (strcmp(vsa, "HRR") == 0 || strcmp(vsa, "HRR_VTB") == 0 || strcmp(vsa, "MBAT") == 0) {
        // convert
        for (i = 0; i < rows; i++) {
            double norm = 0;
            for (j = 0; j < cols; j++) {
                norm += pow(Y[i * cols + j], 2);
            }
            norm = sqrt(norm);
            for (j = 0; j < cols; j++) {
                values[i * cols + j] = Y[i * cols + j] / norm;
            }
        }
    } else if (strcmp(vsa, "FHRR") == 0) {
        // convert
        for (i = 0; i < rows; i++) {
            for (j = 0; j < cols; j++) {
                values[i * cols + j] = atan2(Y[i * cols + j].im, Y[i * cols + j].re);
            }
        }
    } else if (strcmp(vsa, "BSDC") == 0 || strcmp(vsa, "BSDC_SHIFT") == 0 || strcmp(vsa, "BSDC_SEG") == 0) {
        // project values
        get_sLSBH(Y, density, values, rows, cols);
    } else if (strcmp(vsa, "NONE") == 0 || strcmp(vsa, "Proj.") == 0) {
        // use the original vectors without converting
        for (i = 0; i < rows; i++) {
            for (j = 0; j < cols; j++) {
                values[i * cols + j] = Y[i * cols + j];
            }
        }
    } else {
        printf("Representation is not defined!\n");
    }
}

int main() {
    // example usage
    double vsa[] = "MAP_C";
    double Y[] = {1, 2, 3, 4, 5, 6};
    double density = -1;
    double values[6];
    int rows = 2;
    int cols = 3;
    convert_vectors(vsa, Y, density, values, rows, cols);
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%f ", values[i * cols + j]);
        }
        printf("\n");
    }
    return 0;
}

**compute_sim operation:**

The compute_sim function is a versatile utility for calculating similarity between two sets of vectors using various similarity metrics, depending on the specified VSA (Vector Symbolic Architecture) type. It takes two sets of vectors, vectors_1 and vectors_2, and computes a similarity matrix between them.

The similarity calculation method varies based on the chosen VSA type:

- Cosine Similarity (MAP_B, MAP_C, HRR, HRR_VTB, NONE, MAP_I, MBAT, Proj,

- FHRR_cos): For continuous VSAs (MAP_C, HRR), it calculates the cosine similarity between vectors. For binary VSAs (MAP_B, MAP_I), it uses a modified cosine similarity. For complex-valued VSAs (MBAT, FHRR_cos), it computes the average cosine of the angle between vectors.

- Hamming Distance (BSC): For binary sparse coding (BSC), it computes the Hamming distance between binary vectors.

- Overlap (BSDC, BSDC_SHIFT, BSDC_25, BSDC_SEG, BSDC_THIN): For various forms of binary sparse distributed representations (BSDC), it calculates the overlap between binary vectors.
Average Cosine of Distance (FHRR_fft, FHRR):

- For Fourier-based Holographic Reduced Representations (FHRR), it computes the average cosine of the distance between complex-valued vectors.

The function first transposes the input arrays to ensure proper alignment for the similarity calculations. It then initializes a similarity matrix and proceeds to calculate the similarity values based on the selected VSA type. The choice of VSA determines which mathematical operation is performed to measure similarity. This function is essential for assessing the similarity between vectors, which is a fundamental operation in many VSA applications.

In [ ]:
#include <stdio.h>
#include <math.h>

void compute_sim(char* vsa, double* vectors_1, double* vectors_2, int rows_1, int cols_1, int rows_2, int cols_2, double* sim_matrix) {
    int i, j;
    double norm_1, norm_2;

    // first transpose the arrays
    double* transposed_1 = (double*)malloc(cols_1 * rows_1 * sizeof(double));
    double* transposed_2 = (double*)malloc(cols_2 * rows_2 * sizeof(double));
    for (i = 0; i < cols_1; i++) {
        for (j = 0; j < rows_1; j++) {
            transposed_1[i * rows_1 + j] = vectors_1[j * cols_1 + i];
        }
    }
    for (i = 0; i < cols_2; i++) {
        for (j = 0; j < rows_2; j++) {
            transposed_2[i * rows_2 + j] = vectors_2[j * cols_2 + i];
        }
    }

    // initialize sim_matrix with zeros
    for (i = 0; i < rows_1; i++) {
        for (j = 0; j < rows_2; j++) {
            sim_matrix[i * rows_2 + j] = 0.0;
        }
    }

    switch (vsa) {
        case "MAP_B":
        case "MAP_C":
        case "HRR":
        case "HRR_VTB":
        case "NONE":
        case "MAP_I":
        case "MBAT":
        case "Proj":
        case "FHRR_cos":
            // cosine similarity
            for (i = 0; i < rows_1; i++) {
                norm_1 = 0.0;
                for (j = 0; j < cols_1; j++) {
                    norm_1 += transposed_1[i * cols_1 + j] * transposed_1[i * cols_1 + j];
                }
                norm_1 = sqrt(norm_1);

                for (j = 0; j < rows_2; j++) {
                    norm_2 = 0.0;
                    for (int k = 0; k < cols_2; k++) {
                        norm_2 += transposed_2[j * cols_2 + k] * transposed_2[j * cols_2 + k];
                    }
                    norm_2 = sqrt(norm_2);

                    for (int k = 0; k < cols_1; k++) {
                        sim_matrix[i * rows_2 + j] += (transposed_1[i * cols_1 + k] / norm_1) * (transposed_2[j * cols_2 + k] / norm_2);
                    }
                }
            }
            break;
        case "BSC":
            // hamming distance
            for (i = 0; i < rows_1; i++) {
                for (j = 0; j < rows_2; j++) {
                    sim_matrix[i * rows_2 + j] = 1.0 - pdist2(transposed_1 + i * cols_1, transposed_2 + j * cols_2, "hamming");
                }
            }
            break;
        case "BSDC":
        case "BSDC_SHIFT":
        case "BSDC_25":
        case "BSDC_SEG":
        case "BSDC_THIN":
            // overlap
            for (i = 0; i < rows_1; i++) {
                for (j = 0; j < rows_2; j++) {
                    for (int k = 0; k < cols_1; k++) {
                        transposed_1[i * cols_1 + k] = transposed_1[i * cols_1 + k] > 0.0 ? 1.0 : 0.0;
                    }
                    for (int k = 0; k < cols_2; k++) {
                        transposed_2[j * cols_2 + k] = transposed_2[j * cols_2 + k] > 0.0 ? 1.0 : 0.0;
                    }
                    for (int k = 0; k < cols_1; k++) {
                        sim_matrix[i * rows_2 + j] += transposed_1[i * cols_1 + k] * transposed_2[j * cols_2 + k];
                    }
                }
            }
            break;
        case "FHRR_fft":
        case "FHRR":
            // average of cosine of distance
            for (i = 0; i < rows_1; i++) {
                for (j = 0; j < rows_2; j++) {
                    sim_matrix[i * rows_2 + j] = 0.0;
                    for (int k = 0; k < cols_1; k++) {
                        sim_matrix[i * rows_2 + j] += cos(transposed_1[i * cols_1 + k] - transposed_2[j * cols_2 + k]);
                    }
                    sim_matrix[i * rows_2 + j] /= cols_1;
                }
            }
            break;
        default:
            printf("Representation is not defined!\n");
    }

    free(transposed_1);
    free(transposed_2);
}

int main() {
    // example usage
    double vectors_1[3][2] = {{1.0, 2.0}, {3.0, 4.0}, {5.0, 6.0}};
    double vectors_2[2][2] = {{7.0, 8.0}, {9.0, 10.0}};
    double sim_matrix[3][2];

    compute_sim("MAP_B", vectors_1, vectors_2, 3, 2, 2, 2, sim_matrix);

    printf("Similarity Matrix:\n");
    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 2; j++) {
            printf("%f ", sim_matrix[i][j]);
        }
        printf("\n");
    }

    return 0;
}

**complex_sim operation:**


The complexSim function calculates the similarity between two sets of complex-valued vectors, typically used in the context of Fourier-based Holographic Reduced Representations (FHRR). It computes the similarity by measuring the cosine of the angle between pairs of complex numbers in the vectors. The resulting similarity matrix indicates how similar the corresponding pairs of vectors are, with higher values indicating greater similarity.

In [ ]:
#include <stdio.h>
#include <math.h>

void complexSim(double v1[][2], double v2[][2], int size1, int size2, double sim_matrix[][size2]) {
    for (int i = 0; i < size1; i++) {
        for (int j = 0; j < size2; j++) {
            double sum = 0;
            for (int k = 0; k < 2; k++) {
                sum += cos(v1[i][k] - v2[j][k]);
            }
            sim_matrix[i][j] = sum / 2;
        }
    }
}

int main() {
    double v1[3][2] = {{1.0, 2.0}, {3.0, 4.0}, {5.0, 6.0}};
    double v2[2][2] = {{7.0, 8.0}, {9.0, 10.0}};
    double sim_matrix[3][2];

    complexSim(v1, v2, 3, 2, sim_matrix);

    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 2; j++) {
            printf("%f ", sim_matrix[i][j]);
        }
        printf("\n");
    }

    return 0;
}

**cdt operation:**

The cdt function performs a Convolutive Displacement Transformation (CDT) operation on a superposition vector. This operation aims to reduce the density of activated elements in the superposition vector. It iteratively shifts the vector, performs element-wise logical AND with the original vector, and sets elements to zero based on certain criteria. This process continues until the mean density of activated elements across columns falls below a specified threshold (density) or until a maximum number of iterations (max_iters) is reached. The result is a modified superposition vector with reduced density, which can be useful for various operations in vector symbolic architectures.

In [ ]:
#include <stdio.h>
#include <stdlib.h>

int* cdt(int* superposition_vector, int max_iters, double density) {
    // apply CDT procedure
    int* Z = (int*)malloc(sizeof(int) * size);
    for (int i = 0; i < size; i++) {
        Z[i] = superposition_vector[i];
    }
    int counter = 1;
    while (mean(sum(Z) / size(Z, 1)) > density) {
        int r = counter; // detemine the shifting
        int* permutation = circshift(superposition_vector, r);
        int* thinned = and(superposition_vector, permutation);
        for (int i = 0; i < size; i++) {
            if (thinned[i]) {
                Z[i] = 0;
            }
        }
        if (counter > max_iters) { // if more than max_iters iteration, break loop
            break;
        }
        counter++;
    }
    return Z;
}

**bundle_vectors operation:**

The bundle_vectors function combines two sets of vectors (vectors_1 and vectors_2) into a bundled vector representation based on a specified Vector Symbolic Architecture (VSA) method (vsa). It offers various methods for bundling vectors:

- For methods like 'MAP_B', 'MAP_C', 'MAP_I', and 'BSC', it combines vectors based on majority rules, sum, or hamming distance, with optional normalization and density control.

- 'BSDC' performs element-wise disjunction with optional normalization and density control.

- 'HRR', 'HRR_VTB', and 'MBAT' methods add vectors element-wise, optionally normalizing them.

- 'FHRR' computes the average angle between vectors.

- 'BSDC_SHIFT' performs element-wise disjunction with a density-based selection of elements.

- 'BSDC_SEG' bundles vectors in segments with optional normalization and density control.

The resulting bundled vector representation is returned, which can be used in various vector symbolic operations.

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

float* bundle_vectors(char* vsa, float* vectors_1, float* vectors_2, int num_vectors, int vector_dim, int normalize, float density, float max_density) {
    int i, j, k;
    int dim = vector_dim;
    int n_dim = 2;
    float* vector_array = (float*)malloc(sizeof(float) * num_vectors * vector_dim * 2);
    float* bundled_vectors = (float*)malloc(sizeof(float) * num_vectors * vector_dim);
    float* values = (float*)malloc(sizeof(float) * num_vectors);
    float* random_choice = (float*)malloc(sizeof(float) * dim);
    float thresh;
    int number_vec;
    int* idx = (int*)malloc(sizeof(int) * num_vectors);
    int num_segments;
    int size_segments;
    float* values_segments;

    // concatenate the two input vectors
    for (i = 0; i < num_vectors; i++) {
        for (j = 0; j < vector_dim; j++) {
            vector_array[i * vector_dim * 2 + j] = vectors_1[i * vector_dim + j];
            vector_array[i * vector_dim * 2 + j + vector_dim] = vectors_2[i * vector_dim + j];
        }
    }

    switch (vsa) {
        case "MAP_B":
            // majority rule
            for (i = 0; i < num_vectors; i++) {
                values[i] = 0;
                for (j = 0; j < vector_dim; j++) {
                    values[i] += vector_array[i * vector_dim * 2 + j];
                }
                if (normalize) {
                    if (values[i] < -1) {
                        values[i] = -1;
                    }
                    if (values[i] > 1) {
                        values[i] = 1;
                    }
                    random_choice[i] = (float)(rand() % 2) * 2 - 1;
                    if (values[i] == 0) {
                        values[i] = random_choice[i];
                    }
                }
            }
            for (i = 0; i < num_vectors; i++) {
                bundled_vectors[i] = values[i];
            }
            break;
        case "MAP_C":
            for (i = 0; i < num_vectors; i++) {
                values[i] = 0;
                for (j = 0; j < vector_dim; j++) {
                    values[i] += vector_array[i * vector_dim * 2 + j];
                }
                if (normalize) {
                    if (values[i] > 1) {
                        values[i] = 1;
                    }
                    if (values[i] < -1) {
                        values[i] = -1;
                    }
                }
            }
            for (i = 0; i < num_vectors; i++) {
                bundled_vectors[i] = values[i];
            }
            break;
        case "MAP_I":
            for (i = 0; i < num_vectors; i++) {
                values[i] = 0;
                for (j = 0; j < vector_dim; j++) {
                    values[i] += vector_array[i * vector_dim * 2 + j];
                }
            }
            for (i = 0; i < num_vectors; i++) {
                bundled_vectors[i] = values[i];
            }
            break;
        case "BSC":
            for (i = 0; i < num_vectors; i++) {
                values[i] = 0;
                for (j = 0; j < vector_dim; j++) {
                    values[i] += vector_array[i * vector_dim * 2 + j];
                }
                if (normalize) {
                    number_vec = num_vectors;
                    if (number_vec % 2 == 0) {
                        random_choice[i] = (float)(rand() % 2);
                        values[i] += random_choice[i];
                        number_vec++;
                    }
                    thresh = number_vec / 2;
                    if (fabs(thresh - (values[i] / vector_dim)) > 2) {
                        thresh = values[i] / vector_dim;
                    }
                    values[i] = (float)(values[i] / vector_dim > thresh);
                }
            }
            for (i = 0; i < num_vectors; i++) {
                bundled_vectors[i] = values[i];
            }
            break;
        case "BSDC":
            k = floor(max_density * vector_dim);
            for (i = 0; i < num_vectors; i++) {
                values[i] = 0;
                for (j = 0; j < vector_dim; j++) {
                    values[i] += vector_array[i * vector_dim * 2 + j];
                }
            }
            if (normalize) {
                for (i = 0; i < num_vectors; i++) {
                    bundled_vectors[i] = 0;
                }
                for (i = 0; i < k; i++) {
                    idx[i] = i;
                }
                for (i = k; i < num_vectors; i++) {
                    for (j = 0; j < k; j++) {
                        if (values[i] > values[idx[j]]) {
                            idx[j] = i;
                            break;
                        }
                    }
                }
                for (i = 0; i < k; i++) {
                    bundled_vectors[idx[i]] = (float)(values[idx[i]] > 0);
                }
            } else {
                for (i = 0; i < num_vectors; i++) {
                    bundled_vectors[i] = (float)(values[i] >= 1);
                }
            }
            break;
        case "HRR":
        case "HRR_VTB":
        case "MBAT":
            for (i = 0; i < num_vectors; i++) {
                values[i] = 0;
                for (j = 0; j < vector_dim; j++) {
                    values[i] += vector_array[i * vector_dim * 2 + j];
                }
                if (normalize) {
                    values[i] = values[i] / sqrt(values[i] * values[i]);
                }
            }
            for (i = 0; i < num_vectors; i++) {
                bundled_vectors[i] = values[i];
            }
            break;
        case "FHRR":
            for (i = 0; i < num_vectors; i++) {
                values[i] = 0;
                for (j = 0; j < vector_dim; j++) {
                    values[i] += cos(vector_array[i * vector_dim * 2 + j]);
                }
            }
            for (i = 0; i < num_vectors; i++) {
                bundled_vectors[i] = values[i];
            }
            break;
        case "BSDC_SHIFT":
            k = floor(max_density * vector_dim);
            for (i = 0; i < num_vectors; i++) {
                values[i] = 0;
                for (j = 0; j < vector_dim; j++) {
                    values[i] += vector_array[i * vector_dim * 2 + j];
                }
            }
            if (normalize) {
                for (i = 0; i < num_vectors; i++) {
                    bundled_vectors[i] = 0;
                }
                for (i = 0; i < k; i++) {
                    idx[i] = i;
                }
                for (i = k; i < num_vectors; i++) {
                    for (j = 0; j < k; j++) {
                        if (values[i] > values[idx[j]]) {
                            idx[j] = i;
                            break;
                        }
                    }
                }
                for (i = 0; i < k; i++) {
                    bundled_vectors[idx[i]] = (float)(values[idx[i]] > 0);
                }
            } else {
                for (i = 0; i < num_vectors; i++) {
                    bundled_vectors[i] = (float)(values[i] > 0);
                }
            }
            break;
        case "BSDC_SEG":
            num_segments = floor(dim * density);
            size_segments = floor(dim / num_segments);
            k = floor(max_density * size_segments);
            for (i = 0; i < num_vectors; i++) {
                values[i] = 0;
                for (j = 0; j < vector_dim; j++) {
                    values[i] += vector_array[i * vector_dim * 2 + j];
                }
            }
            if (normalize) {
                values_segments = (float*)malloc(sizeof(float) * size_segments * num_segments);
                for (i = 0; i < size_segments * num_segments; i++) {
                    values_segments[i] = values[i];
                }
                for (i = 0; i < k; i++) {
                    idx[i] = i;
                }
                for (i = k; i < size_segments * num_segments; i++) {
                    for (j = 0; j < k; j++) {
                        if (values_segments[i] > values_segments[idx[j]]) {
                            idx[j] = i;
                            break;
                        }
                    }
                }
                for (i = 0; i < k; i++) {
                    bundled_vectors[idx[i]] = (float)(values_segments[idx[i]] > 0);
                }
                for (i = 0; i < size_segments * num_segments; i++) {
                    bundled_vectors[i] = bundled_vectors[i] > 0;
                }
                free(values_segments);
            } else {
                for (i = 0; i < num_vectors; i++) {
                    bundled_vectors[i] = values[i];
                }
            }
            if (size_segments * num_segments != num_vectors) {
                for (i = size_segments * num_segments; i < num_vectors; i++) {
                    bundled_vectors[i] = values[i];
                }
            }
            break;
        default:
            printf("Representation is not defined!\n");
            break;
    }

    free(vector_array);
    free(values);
    free(random_choice);
    free(idx);

    return bundled_vectors;
}

**bind_vectors operation:**

The bind_vectors function performs vector binding operations based on the specified Vector Symbolic Architecture (VSA) method (vsa). It combines two sets of vectors (vectors_1 and vectors_2) into a single bound vector representation. Here's a brief description of what it does for each method:

- For 'MAP_B', 'MAP_C', and 'MAP_I', it performs element-wise multiplication between the vectors.

- 'BSC' efficiently computes XOR operations between vectors.

- 'BSDC' combines vectors using the CDT (Conscious Dynamic Thinning) algorithm, which involves iterative thinning to control density.

- 'BSDC_SHIFT' calculates the shift number based on the dot product with a range of values and then shifts the columns of the second set of vectors accordingly.

- 'HRR' uses circular convolution to bind vectors.

- 'FHRR' computes the binding by taking the modulus of the sum of angles.

- 'BSDC_SEG' binds vectors in segments, where each segment is bound independently.

The resulting bound vector representation is returned for further use in vector symbolic operations.

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

double* bind_vectors(char vsa[], double* vectors_1, double* vectors_2, int num_vectors, int vector_dim, double density, double M) {
    double* bound_vectors = (double*)malloc(num_vectors * vector_dim * sizeof(double));
    int i, j, k;
    if (strcmp(vsa, "MAP_B") == 0 || strcmp(vsa, "MAP_C") == 0 || strcmp(vsa, "MAP_I") == 0) {
        for (i = 0; i < num_vectors; i++) {
            for (j = 0; j < vector_dim; j++) {
                bound_vectors[i * vector_dim + j] = vectors_1[i * vector_dim + j] * vectors_2[i * vector_dim + j];
            }
        }
    }
    else if (strcmp(vsa, "BSC") == 0) {
        for (i = 0; i < num_vectors; i++) {
            for (j = 0; j < vector_dim; j++) {
                bound_vectors[i * vector_dim + j] = (vectors_1[i * vector_dim + j] + vectors_2[i * vector_dim + j] == 1) ? 1.0 : 0.0;
            }
        }
    }
    else if (strcmp(vsa, "BSDC") == 0) {
        double* values_disj = (double*)malloc(num_vectors * vector_dim * sizeof(double));
        for (i = 0; i < num_vectors; i++) {
            for (j = 0; j < vector_dim; j++) {
                values_disj[i * vector_dim + j] = vectors_1[i * vector_dim + j] + vectors_2[i * vector_dim + j];
            }
        }
        double* Z = (double*)malloc(num_vectors * vector_dim * sizeof(double));
        for (i = 0; i < num_vectors; i++) {
            for (j = 0; j < vector_dim; j++) {
                double sum = 0.0;
                for (k = 0; k < vector_dim; k++) {
                    sum += values_disj[i * vector_dim + k] * exp(-pow(j - k, 2) / (2 * pow(density, 2)));
                }
                Z[i * vector_dim + j] = sum;
            }
        }
        for (i = 0; i < num_vectors; i++) {
            for (j = 0; j < vector_dim; j++) {
                bound_vectors[i * vector_dim + j] = Z[i * vector_dim + j];
            }
        }
        free(values_disj);
        free(Z);
    }
    else if (strcmp(vsa, "BSDC_SHIFT") == 0) {
        int* idx = (int*)malloc(num_vectors * sizeof(int));
        for (i = 0; i < num_vectors; i++) {
            int sum = 0;
            for (j = 0; j < vector_dim; j++) {
                sum += (j + 1) * vectors_1[i * vector_dim + j];
            }
            idx[i] = sum;
        }
        for (i = 0; i < num_vectors; i++) {
            for (j = 0; j < vector_dim; j++) {
                bound_vectors[i * vector_dim + j] = vectors_2[i * vector_dim + (j + idx[i]) % vector_dim];
            }
        }
        free(idx);
    }
    else if (strcmp(vsa, "HRR") == 0) {
        double* ccirc = (double*)malloc(num_vectors * vector_dim * sizeof(double));
        for (i = 0; i < num_vectors; i++) {
            for (j = 0; j < vector_dim; j++) {
                double sum_real = 0.0;
                double sum_imag = 0.0;
                for (k = 0; k < vector_dim; k++) {
                    sum_real += vectors_1[i * vector_dim + k] * vectors_2[i * vector_dim + (j - k + vector_dim) % vector_dim];
                    sum_imag += vectors_1[i * vector_dim + k] * vectors_2[i * vector_dim + (j - k + vector_dim) % vector_dim];
                }
                ccirc[i * vector_dim + j] = sum_real + sum_imag;
            }
        }
        for (i = 0; i < num_vectors; i++) {
            for (j = 0; j < vector_dim; j++) {
                bound_vectors[i * vector_dim + j] = ccirc[i * vector_dim + j];
            }
        }
        free(ccirc);
    }
    else if (strcmp(vsa, "HRR_VTB") == 0) {
        double* bound_vectors_part2 = (double*)malloc((vector_dim - sqrt(vector_dim)) * num_vectors * sizeof(double));
        for (i = 0; i < num_vectors; i++) {
            double* V_x = (double*)malloc(vector_dim * vector_dim * sizeof(double));
            for (j = 0; j < vector_dim; j++) {
                for (k = 0; k < vector_dim; k++) {
                    V_x[j * vector_dim + k] = 0.0;
                }
            }
            for (j = 0; j < sqrt(vector_dim); j++) {
                for (k = 0; k < sqrt(vector_dim); k++) {
                    V_x[j * sqrt(vector_dim) + k] = vectors_1[i * vector_dim + j * sqrt(vector_dim) + k];
                }
            }
            for (j = 0; j < num_vectors; j++) {
                double sum = 0.0;
                for (k = 0; k < vector_dim; k++) {
                    sum += V_x[j * vector_dim + k] * vectors_2[i * vector_dim + k];
                }
                bound_vectors[i * vector_dim + j] = sqrt(sqrt(vector_dim)) * sum;
            }
            free(V_x);
        }
        for (i = 0; i < (vector_dim - sqrt(vector_dim)) * num_vectors; i++) {
            bound_vectors[i + num_vectors * vector_dim] = bound_vectors_part2[i];
        }
        free(bound_vectors_part2);
    }
    else if (strcmp(vsa, "FHRR") == 0) {
        for (i = 0; i < num_vectors; i++) {
            for (j = 0; j < vector_dim; j++) {
                bound_vectors[i * vector_dim + j] = fmod(vectors_1[i * vector_dim + j] + vectors_2[i * vector_dim + j] + M, 2 * M) - M;
            }
        }
    }
    else if (strcmp(vsa, "BSDC_SEG") == 0) {
        int num_segments = floor(vector_dim * density);
        int size_segments = floor(vector_dim / num_segments);
        double* role = (double*)malloc(num_segments * size_segments * num_vectors * sizeof(double));
        double* filler = (double*)malloc(num_segments * size_segments * num_vectors * sizeof(double));
        for (i = 0; i < num_segments * size_segments * num_vectors; i++) {
            role[i] = vectors_1[i];
            filler[i] = vectors_2[i];
        }
        double* role_segments = (double*)malloc(size_segments * num_segments * num_vectors * sizeof(double));
        double* filler_segments = (double*)malloc(size_segments * num_segments * num_vectors * sizeof(double));
        for (i = 0; i < size_segments * num_segments * num_vectors; i++) {
            role_segments[i] = role[i];
            filler_segments[i] = filler[i];
        }
        int* role_idx = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* filler_idx = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        for (i = 0; i < size_segments * num_segments * num_vectors; i++) {
            role_idx[i] = (role_segments[i] != 0.0) ? 1 : 0;
            filler_idx[i] = (filler_segments[i] != 0.0) ? 1 : 0;
        }
        int* role_rows = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* filler_rows = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* filler_cols = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        int* filler_tables = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        for (i = 0; i < size_segments * num_segments * num_vectors; i++) {
            role_rows[i] = i / (size_segments * num_segments);
            filler_rows[i] = i / (size_segments * num_segments);
            filler_cols[i] = i % size_segments;
            filler_tables[i] = i % num_segments;
        }
        int* result_rows = (int*)malloc(size_segments * num_segments * num_vectors * sizeof(int));
        for (i = 0; i < size_segments * num_segments * num_vectors; i++) {
            result_rows[i] = (role_rows[filler_cols[i] * filler_tables[i]] + filler_rows[i] - 1) % size_segments + 1;
        }
        for (i = 0; i < size_segments * num_segments * num_vectors; i++) {
            bound_vectors[i] = 0.0;
        }
        for (i = 0; i < size_segments * num_segments * num_vectors; i++) {
            int idx = result_rows[i] * size_segments * num_segments + filler_cols[i] * num_segments + filler_tables[i];
            bound_vectors[idx] = 1.0;
        }
        for (i = 0; i < size_segments * num_segments * num_vectors; i++) {
            bound_vectors[i + size_segments * num_segments * num_vectors] = filler[i];
        }
        free(role);
        free(filler);
        free(role_segments);
        free(filler_segments);
        free(role_idx);
        free(filler_idx);
        free(role_rows);
        free(filler_rows);
        free(filler_cols);
        free(filler_tables);
        free(result_rows);
    }
    else if (strcmp(vsa, "MBAT") == 0) {
        for (i = 0; i < num_vectors; i++) {
            double* M_ = (double*)malloc(vector_dim * vector_dim * sizeof(double));
            for (j = 0; j < vector_dim; j++) {
                for (k = 0; k < vector_dim; k++) {
                    M_[j * vector_dim + k] = M;
                }
            }
            int idx = 0;
            for (j = 0; j < vector_dim; j++) {
                idx += (j + 1) * vectors_1[i * vector_dim + j];
            }
            for (j = 0; j < vector_dim; j++) {
                double sum = 0.0;
                for (k = 0; k < vector_dim; k++) {
                    sum += M_[j * vector_dim + k] * vectors_2[i * vector_dim + k];
                }
                bound_vectors[i * vector_dim + j] = sum;
            }
            free(M_);
        }
    }
    else {
        printf("Representation is not defined!\n");
    }
    return bound_vectors;
}

int main() {
    // Example usage
    int num_vectors = 3;
    int vector_dim = 4;
    double* vectors_1 = (double*)malloc(num_vectors * vector_dim * sizeof(double));
    double* vectors_2 = (double*)malloc(num_vectors * vector_dim * sizeof(double));
    double density = 0.25;
    double M = 0.0;
    double* bound_vectors = bind_vectors("MAP_B", vectors_1, vectors_2, num_vectors, vector_dim, density, M);
    for (int i = 0; i < num_vectors; i++) {
        for (int j = 0; j < vector_dim; j++) {
            printf("%f ", bound_vectors[i * vector_dim + j]);
        }
        printf("\n");
    }
    free(vectors_1);
    free(vectors_2);
    free(bound_vectors);
    return 0;
}

**vsa_env operation:**

The vsa_env class is a Python class that provides various Vector Symbolic Architecture (VSA) operations, and it can be used to work with different VSA implementations. Here's an overview of its key methods and functionalities:

- __init__: Initializes the VSA environment with specified parameters such as VSA type (vsa), dimension (dim), density (density), and maximum density (max_density).

- add_vector: Adds vectors to the environment. It generates random vectors if vec is not provided. Vectors can be associated with names for easy retrieval.

- sim: Computes similarity between two sets of vectors using different similarity metrics based on the selected VSA type.

- bind: Binds two sets of vectors together using the specified VSA type.

- unbind: Unbinds two sets of vectors using the specified VSA type.

- bundle: Bundles two sets of vectors together, with options for normalization and density control.

- permute: Performs vector permutation by rolling the elements of a vector.

- find_k_nearest: Finds the k-nearest vectors in the environment based on similarity.

- find_by_name: Retrieves vectors by their associated names.

- frac_binding: Performs fractional binding on a vector.

- convert: Converts vectors using the selected VSA type.

- generate_vectors: Generates random vectors based on the specified VSA type and parameters.

- compute_sim: Computes similarity between two sets of vectors using different similarity metrics.

- bind_vectors: Binds two sets of vectors together using the specified VSA type and density.

- unbind_vectors: Unbinds two sets of vectors using the specified VSA type and density.

- bundle_vectors: Bundles two sets of vectors together, with options for normalization and density control.

- fractional_binding: Performs fractional binding on a vector.

- convert_vectors: Converts vectors using the selected VSA type and density.

This class supports various VSA implementations and perform operations like binding, unbinding, and similarity computations on sets of vectors.

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>

// Define the vsa_env structure
typedef struct {
    int dim;
    char vsa[10];
    double** item_mem;
    int item_mem_size;
    double density;
    double max_density;
    double** M;
} vsa_env;

// Function to initialize the vsa_env structure
vsa_env* vsa_env_init(char* vsa, int dim, double density, double max_density) {
    // Allocate memory for the vsa_env structure
    vsa_env* env = (vsa_env*)malloc(sizeof(vsa_env));

    // Set the properties of the vsa_env structure
    env->dim = dim;
    strcpy(env->vsa, vsa);
    env->item_mem = NULL;
    env->item_mem_size = 0;
    env->density = density;
    env->max_density = max_density;
    env->M = NULL;

    return env;
}

// Function to add vectors to the item memory
double** add_vector(vsa_env* env, double** vec, int num, char* name, int add_item, int return_vector) {
    // Allocate memory for the new vectors
    double** vectors = (double**)malloc(num * sizeof(double*));
    for (int i = 0; i < num; i++) {
        vectors[i] = (double*)malloc(env->dim * sizeof(double));
    }

    // Generate random vectors if no input vectors are given
    if (vec == NULL) {
        for (int i = 0; i < num; i++) {
            for (int j = 0; j < env->dim; j++) {
                vectors[i][j] = (double)rand() / RAND_MAX;
            }
        }
    } else {
        // Copy the input vectors to the new vectors
        for (int i = 0; i < num; i++) {
            memcpy(vectors[i], vec[i], env->dim * sizeof(double));
        }
    }

    // Add the vectors to the item memory
    if (add_item) {
        env->item_mem_size += num;
        env->item_mem = (double**)realloc(env->item_mem, env->item_mem_size * sizeof(double*));
        for (int i = 0; i < num; i++) {
            env->item_mem[env->item_mem_size - num + i] = vectors[i];
        }

        // Generate names if not given
        if (name == NULL) {
            char** names = (char**)malloc(env->item_mem_size * sizeof(char*));
            for (int i = 0; i < env->item_mem_size - num; i++) {
                names[i] = (char*)malloc(9 * sizeof(char));
                strcpy(names[i], env->item_mem[i]);
            }
            for (int i = env->item_mem_size - num; i < env->item_mem_size; i++) {
                names[i] = (char*)malloc(9 * sizeof(char));
                for (int j = 0; j < 8; j++) {
                    names[i][j] = 'A' + rand() % 26;
                }
                names[i][8] = '\0';
            }
            free(env->item_mem);
            env->item_mem = names;
        } else {
            char** names = (char**)malloc(env->item_mem_size * sizeof(char*));
            for (int i = 0; i < env->item_mem_size - num; i++) {
                names[i] = (char*)malloc(9 * sizeof(char));
                strcpy(names[i], env->item_mem[i]);
            }
            for (int i = env->item_mem_size - num; i < env->item_mem_size; i++) {
                names[i] = (char*)malloc((strlen(name) + 1) * sizeof(char));
                strcpy(names[i], name);
            }
            free(env->item_mem);
            env->item_mem = names;
        }
    }

    // Return vectors only if param is set
    if (!return_vector) {
        for (int i = 0; i < num; i++) {
            free(vectors[i]);
        }
        free(vectors);
        vectors = NULL;
    }

    return vectors;
}

// Function to compute the similarity between two vectors
double** sim(vsa_env* env, double** vectors_1, double** vectors_2) {
    // Compute the similarity matrix
    double** sim_matrix = (double**)malloc(env->item_mem_size * sizeof(double*));
    for (int i = 0; i < env->item_mem_size; i++) {
        sim_matrix[i] = (double*)malloc(env->item_mem_size * sizeof(double));
        for (int j = 0; j < env->item_mem_size; j++) {
            double sum = 0.0;
            for (int k = 0; k < env->dim; k++) {
                sum += vectors_1[i][k] * vectors_2[j][k];
            }
            sim_matrix[i][j] = sum;
        }
    }

    return sim_matrix;
}

// Function to bind vectors_1 and vectors_2
double** bind(vsa_env* env, double** vectors_1, double** vectors_2) {
    // Bind vectors_1 and vectors_2
    double** bound_vectors = (double**)malloc(env->item_mem_size * sizeof(double*));
    for (int i = 0; i < env->item_mem_size; i++) {
        bound_vectors[i] = (double*)malloc(env->dim * sizeof(double));
        for (int j = 0; j < env->dim; j++) {
            bound_vectors[i][j] = vectors_1[i][j] + vectors_2[i][j];
        }
    }

    return bound_vectors;
}

// Function to unbind vectors_1 and vectors_2
double** unbind(vsa_env* env, double** vectors_1, double** vectors_2) {
    // Unbind vectors_1 and vectors_2
    double** unbound_vectors = (double**)malloc(env->item_mem_size * sizeof(double*));
    for (int i = 0; i < env->item_mem_size; i++) {
        unbound_vectors[i] = (double*)malloc(env->dim * sizeof(double));
        for (int j = 0; j < env->dim; j++) {
            unbound_vectors[i][j] = vectors_1[i][j] - vectors_2[i][j];
        }
    }

    return unbound_vectors;
}

// Function to bundle vectors_1 and vectors_2
double** bundle(vsa_env* env, double** vectors_1, double** vectors_2, int normalize) {
    // Bundle vectors_1 and vectors_2
    double** bundled_vectors = (double**)malloc(env->item_mem_size * sizeof(double*));
    for (int i = 0; i < env->item_mem_size; i++) {
        bundled_vectors[i] = (double*)malloc(env->dim * sizeof(double));
        double sum = 0.0;
        for (int j = 0; j < env->dim; j++) {
            bundled_vectors[i][j] = vectors_1[i][j] + vectors_2[i][j];
            sum += bundled_vectors[i][j] * bundled_vectors[i][j];
        }
        if (normalize) {
            double norm = sqrt(sum);
            for (int j = 0; j < env->dim; j++) {
                bundled_vectors[i][j] /= norm;
            }
        }
    }

    return bundled_vectors;
}

// Function to permute the input vector
double* permute(vsa_env* env, double* vector, int p) {
    // Permute the input vector
    double* permuted_vector = (double*)malloc(env->dim * sizeof(double));
    for (int i = 0; i < env->dim; i++) {
        permuted_vector[i] = vector[(i + p) % env->dim];
    }

    return permuted_vector;
}

// Function to find the k best matches in item memory with input vector
double** find_k_nearest(vsa_env* env, double** vectors_in, int k, char*** names, double** s) {
    // Compute the similarity between vectors_in and item memory
    double** sim_vec = sim(env, env->item_mem, vectors_in);

    // Sort the similarity matrix in descending order
    double** sim_vec_sort = (double**)malloc(env->item_mem_size * sizeof(double*));
    for (int i = 0; i < env->item_mem_size; i++) {
        sim_vec_sort[i] = (double*)malloc(env->item_mem_size * sizeof(double));
        memcpy(sim_vec_sort[i], sim_vec[i], env->item_mem_size * sizeof(double));
    }
    int* idx = (int*)malloc(env->item_mem_size * sizeof(int));
    for (int i = 0; i < env->item_mem_size; i++) {
        idx[i] = i;
    }
    for (int i = 0; i < env->item_mem_size; i++) {
        for (int j = i + 1; j < env->item_mem_size; j++) {
            if (sim_vec_sort[i][j] < sim_vec_sort[j][i]) {
                double temp = sim_vec_sort[i][j];
                sim_vec_sort[i][j] = sim_vec_sort[j][i];
                sim_vec_sort[j][i] = temp;
                int temp_idx = idx[i];
                idx[i] = idx[j];
                idx[j] = temp_idx;
            }
        }
    }

    // Get the k nearest neighbors
    double** s_highest = (double**)malloc(k * sizeof(double*));
    for (int i = 0; i < k; i++) {
        s_highest[i] = (double*)malloc(env->item_mem_size * sizeof(double));
        memcpy(s_highest[i], sim_vec_sort[i], env->item_mem_size * sizeof(double));
    }
    int* rows = (int*)malloc(k * sizeof(int));
    for (int i = 0; i < k; i++) {
        rows[i] = idx[i];
    }
    *names = (char**)malloc(k * sizeof(char*));
    for (int i = 0; i < k; i++) {
        (*names)[i] = (char*)malloc(9 * sizeof(char));
        strcpy((*names)[i], env->item_mem[rows[i]]);
    }
    double** vectors = (double**)malloc(k * sizeof(double*));
    for (int i = 0; i < k; i++) {
        vectors[i] = (double*)malloc(env->dim * sizeof(double));
        memcpy(vectors[i], env->item_mem[rows[i]], env->dim * sizeof(double));
    }
    *s = (double*)malloc(k * sizeof(double));
    for (int i = 0; i < k; i++) {
        (*s)[i] = sim_vec[rows[i]][rows[i]];
    }

    // Free memory
    for (int i = 0; i < env->item_mem_size; i++) {
        free(sim_vec_sort[i]);
    }
    free(sim_vec_sort);
    free(idx);
    for (int i = 0; i < env->item_mem_size; i++) {
        free(sim_vec[i]);
    }
    free(sim_vec);

    return vectors;
}

// Function to find the vector by name
double* find_by_name(vsa_env* env, char* vector_name) {
    // Find the vector by name
    double* vector = NULL;
    for (int i = 0; i < env->item_mem_size; i++) {
        if (strcmp(env->item_mem[i], vector_name) == 0) {
            vector = (double*)malloc(env->dim * sizeof(double));
            memcpy(vector, env->item_mem[i], env->dim * sizeof(double));
            break;
        }
    }

    if (vector == NULL) {
        printf("No vector for name %s found!\n", vector_name);
    }

    return vector;
}

// Function to convert the input into the specific range of the corresponding vsa type
double** convert(vsa_env* env, double** vector_array) {
    // Convert the input vectors
    double** converted_vectors = (double**)malloc(env->item_mem_size * sizeof(double*));
    for (int i = 0; i < env->item_mem_size; i++) {
        converted_vectors[i] = (double*)malloc(env->dim * sizeof(double));
        for (int j = 0; j < env->dim; j++) {
            converted_vectors[i][j] = vector_array[i][j] * env->density;
        }
    }

    return converted_vectors;
}

// Function to generate random names
char** rnd_name(int size) {
    // Generate random names
    char** names = (char**)malloc(size * sizeof(char*));
    for (int i = 0; i < size; i++) {
        names[i] = (char*)malloc(9 * sizeof(char));
        for (int j = 0; j < 8; j++) {
            names[i][j] = 'A' + rand() % 26;
        }
        names[i][8] = '\0';
    }

    return names;
}

int main() {
    // Create a vsa_env object
    vsa_env* env = vsa_env_init("MAP_B", 10000, -1, 1);

    // Add vectors to the item memory
    double** vectors = add_vector(env, NULL, 1, NULL, 1, 1);

    // Compute the similarity between two vectors
    double** sim_matrix = sim(env, vectors, vectors);

    // Bind vectors_1 and vectors_2
    double** bound_vectors = bind(env, vectors, vectors);

    // Unbind vectors_1 and vectors_2
    double** unbound_vectors = unbind(env, vectors, vectors);

    // Bundle vectors_1 and vectors_2
    double** bundled_vectors = bundle(env, vectors, vectors, 0);

    // Permute the input vector
    double* permuted_vector = permute(env, vectors[0], 1);

    // Find the k best matches in item memory with input vector
    char** names;
    double* s;
    double** nearest_vectors = find_k_nearest(env, vectors, 1, &names, &s);

    // Find the vector by name
    double* vector = find_by_name(env, names[0]);

    // Convert the input vectors
    double** converted_vectors = convert(env, vectors);

    // Free memory
    for (int i = 0; i < env->item_mem_size; i++) {
        free(vectors[i]);
    }
    free(vectors);
    for (int i = 0; i < env->item_mem_size; i++) {
        free(sim_matrix[i]);
    }
    free(sim_matrix);
    for (int i = 0; i < env->item_mem_size; i++) {
        free(bound_vectors[i]);
    }
    free(bound_vectors);
    for (int i = 0; i < env->item_mem_size; i++) {
        free(unbound_vectors[i]);
    }
    free(unbound_vectors);
    for (int i = 0; i < env->item_mem_size; i++) {
        free(bundled_vectors[i]);
    }
    free(bundled_vectors);
    free(permuted_vector);
    for (int i = 0; i < env->item_mem_size; i++) {
        free(nearest_vectors[i]);
    }
    free(nearest_vectors);
    for (int i = 0; i < env->item_mem_size; i++) {
        free(names[i]);
    }
    free(names);
    free(s);
    free(vector);
    for (int i = 0; i < env->item_mem_size; i++) {
        free(converted_vectors[i]);
    }
    free(converted_vectors);
    for (int i = 0; i < env->item_mem_size; i++) {
        free(env->item_mem[i]);
    }
    free(env->item_mem);
    free(env);

    return 0;
}

**demo_cpu:**

Demonstrates the usage of the Vector Symbolic Architecture (VSA) in Python for various VSA types. It loads images from a directory, converts them to vectors, and performs VSA operations like bundling, binding/unbinding, and vector retrieval. The code iterates through different VSA types, including MAP, BSC, BSDC, HRR, FHRR, and more. It measures the elapsed time for each operation, prints similarity values where applicable, and demonstrates finding vectors using the item memory and handling noise vectors.

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// Function to convert RGB image to grayscale
void rgb2gray(unsigned char *img, int squareSize) {
    int i, j;
    unsigned char r, g, b, gray;
    for (i = 0; i < squareSize; i++) {
        for (j = 0; j < squareSize; j++) {
            r = img[(i * squareSize + j) * 3];
            g = img[(i * squareSize + j) * 3 + 1];
            b = img[(i * squareSize + j) * 3 + 2];
            gray = (unsigned char)(0.2989 * r + 0.5870 * g + 0.1140 * b);
            img[i * squareSize + j] = gray;
        }
    }
}

// Function to reshape image to a vector
void reshape(unsigned char *img, int squareSize, int *imgVector) {
    int i, j;
    for (i = 0; i < squareSize; i++) {
        for (j = 0; j < squareSize; j++) {
            imgVector[i * squareSize + j] = img[i * squareSize + j];
        }
    }
}

int main() {
    // Create the object of a specific VSA type
    char *vsa_types[] = {"MAP_C", "MAP_B", "MAP_I", "BSC", "BSDC", "BSDC_SHIFT", "BSDC_SEG", "HRR", "FHRR"};
    int num_types = sizeof(vsa_types) / sizeof(vsa_types[0]);
    int type_idx, i, j;
    for (type_idx = 0; type_idx < num_types; type_idx++) {
        char *type = vsa_types[type_idx];
        int dim = 1024;
        // Load images from a directory
        char *imageDir = "/MATLAB Drive/VSA_Toolbox-master/+experimental_scripts/GardensPointWalking/night_right";
        // Read image files from the directory
        char **imageFiles;
        int numFiles = 0;
        // Iterate through each image and convert it to a vector
        unsigned char **combinedVector;
        int *combinedVectorSize;
        for (i = 0; i < numFiles / 2; i++) {
            char *currentFileName = imageFiles[i];
            char *currentImagePath = malloc(strlen(imageDir) + strlen(currentFileName) + 2);
            strcpy(currentImagePath, imageDir);
            strcat(currentImagePath, "/");
            strcat(currentImagePath, currentFileName);
            // Read the image
            unsigned char *img = NULL;
            int squareSize = 256;
            rgb2gray(img, squareSize);
            int *imgVector = malloc(squareSize * squareSize * sizeof(int));
            reshape(img, squareSize, imgVector);
            combinedVector[i] = imgVector;
            combinedVectorSize[i] = squareSize * squareSize;
            free(currentImagePath);
        }
        // Add vectors to VSA environment
        int **v_array_1 = malloc(numFiles / 2 * sizeof(int *));
        for (i = 0; i < numFiles / 2; i++) {
            v_array_1[i] = combinedVector[i];
        }
        int **v_array_2 = malloc(numFiles / 2 * sizeof(int *));
        for (i = numFiles / 2; i < numFiles; i++) {
            v_array_2[i - numFiles / 2] = combinedVector[i];
        }
        int **bundle = malloc(numFiles / 2 * sizeof(int *));
        for (i = 0; i < numFiles / 2; i++) {
            bundle[i] = malloc(dim * sizeof(int));
            for (j = 0; j < dim; j++) {
                bundle[i][j] = v_array_1[i][j] + v_array_2[i][j];
            }
        }
        // Unbinding operation
        int **v_array_1_unbind = malloc(numFiles / 2 * sizeof(int *));
        for (i = 0; i < numFiles / 2; i++) {
            v_array_1_unbind[i] = malloc(dim * sizeof(int));
            for (j = 0; j < dim; j++) {
                v_array_1_unbind[i][j] = bundle[i][j] - v_array_2[i][j];
            }
        }
        int **v_array_2_unbind = malloc(numFiles / 2 * sizeof(int *));
        for (i = 0; i < numFiles / 2; i++) {
            v_array_2_unbind[i] = v_array_2[i];
        }
        // Use the item memory to find vectors
        int dim_item = 65536;
        int *v_array_1_item = malloc(dim_item * sizeof(int));
        for (i = 0; i < combinedVectorSize[0]; i++) {
            v_array_1_item[i] = combinedVector[0][i];
        }
        int *v_item = malloc(dim_item * sizeof(int));
        for (i = 0; i < dim_item; i++) {
            v_item[i] = rand() % 2;
        }
        int *v_clean = malloc(dim_item * sizeof(int));
        char *name = malloc(10 * sizeof(char));
        int s;
        for (i = 0; i < dim_item; i++) {
            v_clean[i] = v_array_1_item[i];
        }
        s = 0;
        for (i = 0; i < dim_item; i++) {
            s += v_clean[i] * v_item[i];
        }
        // Convert similarity to percentage for BSDC types
        if (strcmp(type, "BSDC") == 0 || strcmp(type, "BSDC_SHIFT") == 0 || strcmp(type, "BSDC_SEG") == 0) {
            float similarity_decimal = (float)s / dim_item;
            printf("Found noisy vector probe with similarity of %f\n", similarity_decimal);
        } else {
            printf("Found noisy vector %s with similarity of %d\n", name, s);
        }
        // Bundle the probe with noise vector
        int *noise = malloc(dim_item * sizeof(int));
        for (i = 0; i < dim_item; i++) {
            noise[i] = 0;
        }
        int *bundle_probe = malloc(dim_item * sizeof(int));
        for (i = 0; i < dim_item; i++) {
            bundle_probe[i] = v_item[i] + noise[i];
        }
        for (i = 0; i < dim_item; i++) {
            v_clean[i] = bundle_probe[i];
        }
        s = 0;
        for (i = 0; i < dim_item; i++) {
            s += v_clean[i] * v_item[i];
        }
        // Convert similarity to decimal format for BSDC types
        if (strcmp(type, "BSDC") == 0 || strcmp(type, "BSDC_SHIFT") == 0 || strcmp(type, "BSDC_SEG") == 0) {
            float similarity_decimal = (float)s / dim_item;
            printf("Found noisy vector probe with similarity of %f\n", similarity_decimal);
        } else {
            printf("Found noisy vector %s with similarity of %d\n", name, s);
        }
        // Free memory
        for (i = 0; i < numFiles / 2; i++) {
            free(combinedVector[i]);
            free(v_array_1[i]);
            free(v_array_2[i]);
            free(bundle[i]);
            free(v_array_1_unbind[i]);
            free(v_array_2_unbind[i]);
        }
        free(combinedVector);
        free(combinedVectorSize);
        free(imageFiles);
        free(v_array_1);
        free(v_array_2);
        free(bundle);
        free(v_array_1_unbind);
        free(v_array_2_unbind);
        free(v_array_1_item);
        free(v_item);
        free(v_clean);
        free(name);
        free(noise);
        free(bundle_probe);
    }
    return 0;
}